In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from typing import TypedDict, List, Optional
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

ImportError: cannot import name 'RootModel' from 'pydantic' (d:\skala_gai\venv\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)

In [139]:
from GraphState import GraphState

In [140]:
# LLM 정의
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",  # 또는 "gpt-3.5-turbo"
    temperature=0.3
)

In [141]:
# 프롬프트 템플릿 정의
investment_prompt = PromptTemplate.from_template("""
다음은 특정 스타트업에 대한 3가지 분석 보고서입니다.

[기술 분석 보고서]
{tech_report}

[경쟁사 비교 보고서]
{competitor_report}

[시장 분석 보고서]
{market_report}

위의 정보를 바탕으로, 아래 3가지 항목을 평가해 주세요:
회사명: (스타트업의 회사명)

1. **시장성 평가**
   - 점수 (0~10점)
   - 설명 (시장 성장 가능성, 진입장벽, 수요 등 포함)

2. **제품 기술력 평가**
   - 점수 (0~10점)
   - 설명 (핵심 기술, 독창성, 확장성 등 포함)

3. **경쟁 우위 평가**
   - 점수 (0~10점)
   - 설명 (경쟁사 대비 차별성, 지속 가능성 등 포함)

---

4. 최종 평가

- 최종 점수 = 시장성*0.5 + 기술력*0.3 + 경쟁우위*0.2 (소수점 둘째 자리까지)
- 총평: 전반적인 투자 판단과 함께, 고려할 만한 리스크 요인 등을 간단히 요약
""")

In [142]:
# 에이전트 함수 정의
def investment_analysis_agent(state: GraphState) -> GraphState:
    prompt = investment_prompt.format(
        tech_report=state["tech_report"],
        competitor_report=state["competitor_report"],
        market_report=state["market_report"],
    )
    investment_summary_msg = llm.invoke(prompt)
    investment_summary = investment_summary_msg.content
    report = f"""
    [{state["current_company"]} 보고서]

    A. 기술 분석
    {state["tech_report"]}

    B. 경쟁사 비교
    {state["competitor_report"]}

    C. 시장 분석
    {state["market_report"]}

    D. 투자 평가
    {investment_summary}
    """
    return {
        **state,
        "investment_summary": report
    }

In [143]:
def validate_report(state: GraphState) -> str:
    summary = state["investment_summary"]

    eval_prompt = f"""
당신은 벤처 캐피탈의 투자 심사관으로, 아래 투자 분석 보고서가 출력 형식과 품질 기준을 잘 따르고 있는지 평가해야 합니다.

투자 분석 보고서:

------------------------
{summary}
------------------------

다음 기준에 따라 판단하세요:

1. **시장성 평가, 제품 기술력 평가, 경쟁 우위 평가 항목의 점수가 0~10 사이의 정수인지**
2. **시장성 평가, 제품 기술력 평가, 경쟁 우위 평가 항목의 설명이 해당 점수에 대한 타당한 설명으로 충분한지**
3. **최종 평가의 점수가 0~10 사이의 소수(float)인지**
4. **최종평가가 종합적인 평가로서 자연스럽고 논리적인지**

판단 결과는 아래 기준에 따라 선택하세요:

- **PASS**: 모든 항목의 형식과 설명이 정확하며 논리적으로 납득 가능함
- **RETRY**: 일부 설명이 부족하거나 점수가 애매하지만 수정 가능함
- **FAIL**: 형식 오류나 설명 누락 등으로 활용 불가능함

단 한 단어만 출력하세요: `PASS`, `RETRY`, `FAIL`
"""

    judgment = llm.invoke(eval_prompt).content.strip().upper()

    if judgment not in ["PASS", "RETRY", "FAIL"]:
        print(state.get("investment_summary_retry_count", 0))
        judgment = "RETRY" if state.get("investment_summary_retry_count", 0) < 1 else "FAIL"

    return judgment

In [144]:
# 재시도 시 카운트 증가
def increment_retry(state: GraphState) -> GraphState:
    
    return {**state, "investment_summary_retry_count": state.get("investment_summary_retry_count", 0) + 1}

In [145]:
def validate_final_report(report: str, retry_count: int = 0) -> str:
    eval_prompt = f"""
당신은 벤처 캐피탈의 투자 심사관으로, 아래 투자 분석 보고서가 출력 형식과 품질 기준을 잘 따르고 있는지 평가해야 합니다.

투자 분석 보고서:

------------------------
{report}
------------------------

다음 기준에 따라 판단하세요:

1. **시장성 평가, 제품 기술력 평가, 경쟁 우위 평가 항목의 점수가 0~10 사이의 정수인지**
2. **각 항목에 대한 설명이 해당 점수에 대한 타당한 설명으로 충분한지**
3. **최종 평가의 점수가 0~10 사이의 소수(float)인지**
4. **최종평가가 종합적인 평가로서 자연스럽고 논리적인지**

판단 결과는 아래 중 하나로 선택하세요:

- **PASS**: 모든 항목의 형식과 설명이 정확하며 논리적으로 납득 가능함
- **RETRY**: 일부 설명이 부족하거나 점수가 애매하지만 수정 가능함
- **FAIL**: 형식 오류나 설명 누락 등으로 활용 불가능함

단 한 단어만 출력하세요: `PASS`, `RETRY`, `FAIL`
"""

    judgment = llm.invoke(eval_prompt).content.strip().upper()
    if judgment not in ["PASS", "RETRY", "FAIL"]:
        judgment = "RETRY" if retry_count < 2 else "FAIL"
    return judgment

In [146]:
def save_markdown(text: str, filename: str = "투자_최종_보고서.md", silent: bool = False):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)
    if not silent:
        print(f"✅ Markdown 파일 저장 완료: {filename}")


def final_report_agent_with_state(state: GraphState, max_retries: int = 3) -> dict:
    max_retries: int = 3
    reports = state.get("reports")
    full_report = ""
    for text in reports:
        full_report = full_report + "\n" + text

    for i in range(max_retries):
        prompt = f"""
    당신은 벤처캐피탈의 투자 분석 보고서 작성 전문가입니다.

    아래는 AI 분석 에이전트들이 생성한 평가 결과를 바탕으로 구성된 텍스트입니다.
    이 텍스트를 기반으로 다음 기준을 모두 충족하는 **최종 종합 보고서**를 작성해주세요:

    1. **각 기업에 대해 점수(시장성, 기술력, 경쟁력, 최종점수)와 설명이 3줄 이상 명확하게 정리**되어야 합니다.
    2. **최종점수 기준으로 기업들을 투자 우선순위에 따라 정렬**해야 하며 각 기업별 최종점수를 명시해주어야 합니다.
    3. **각 기업별 투자 추천 여부 및 이유가 시장성, 기술력, 경쟁력의 요소를 바탕으로 명확하고 자세하게 작성**되어야 합니다.
    4. **전체 산업/기술 트렌드 분석, 공통 리스크 요인, 향후 투자 전략 제안이 포함**되어야 합니다.
    5. 모든 내용은 벤처캐피탈 보고서에 적합하도록 **논리적이고 명료한 문단 구성**으로 작성해주세요.

    ------------------------
    {full_report}
    ------------------------
    """

        # 보고서 생성
        final_report = llm.invoke(prompt).content.strip()

    # 저장은 마지막에만, 메시지도 여기서만 출력
    save_markdown(final_report, silent=False)

    return {
        **state,
        "final_report": final_report,
    }

In [147]:
from MarketReportAgent import market_agent
from CompetitorReportAgent import competitor_agent
from TechReportAgent import tech_agent

In [148]:
STARTUP_LIST = ["업스테이지", "노타AI", "트웰브랩스", "뤼이드", "에어스메디컬"]

In [149]:
from langgraph.graph import StateGraph, END

def role_dispatch_agent(state: GraphState) -> GraphState:
    idx = state["current_index"]
    company = STARTUP_LIST[idx]
    return {
        "current_index": idx,
        "current_company": company
    }

def increment_index(state: GraphState) -> GraphState:
    summary = state.get("investment_summary", "[요약 없음]")
    reports = state.get("reports", [])
    print(state["current_index"])
    return {
        **state,
        "current_index": state["current_index"] + 1,
        "tech_report": None,
        "competitor_report": None,
        "market_report": None,
        "investment_summary": None,
        "investment_summary_retry_count": 0,
        "reports": reports + [summary]
    }

def final_report_agent(state: GraphState) -> GraphState:
    all_reports = "\n\n".join(state["reports"])
    prompt = f"""
다음은 다섯 개 기업에 대한 투자 판단 보고서입니다:

{all_reports}

이 보고서를 바탕으로 전체적인 투자 전략 및 종합 의견을 작성해주세요.
"""
    final = llm.invoke(prompt).content
    return {
        **state,
        "final_report": final
    }

def check_continue(state: GraphState) -> str:
    return "continue" if state["current_index"] < len(STARTUP_LIST) else "done"
# ----------------------------------------
# LangGraph 구성
# ----------------------------------------

builder = StateGraph(GraphState)

# 노드 등록
builder.add_node("dispatch", RunnableLambda(role_dispatch_agent))
builder.add_node("tech", RunnableLambda(tech_agent))
builder.add_node("competitor", RunnableLambda(competitor_agent))
builder.add_node("market", RunnableLambda(market_agent))
# builder.add_node("join", join(["dispatch", "tech", "competitor", "market"]))
builder.add_node("investment_report", RunnableLambda(investment_analysis_agent))
builder.add_node("increment_retry", RunnableLambda(increment_retry))
builder.add_node("final", RunnableLambda(final_report_agent_with_state))
builder.add_node("increment_index", RunnableLambda(increment_index))
builder.add_node("check_ready", RunnableLambda(lambda x: x))

# 병렬 실행처럼 동작하도록 edge 연결 (각각 따로)
builder.set_entry_point("dispatch")
builder.add_edge("dispatch", "tech")
builder.add_edge("tech", "competitor")
builder.add_edge("competitor", "market")
builder.add_edge("market", "investment_report")

builder.add_conditional_edges("investment_report", validate_report, {
    "PASS": "increment_index",
    "RETRY": "increment_retry",
    "FAIL": "increment_index"
})
builder.add_edge("increment_retry", "investment_report")
builder.add_conditional_edges("increment_index", check_continue, {
    "continue": "dispatch",
    "done": "final"
})
builder.add_edge("final", END)

graph = builder.compile()


In [150]:
STARTUP_LIST = ["업스테이지", "노타AI", "트웰브랩스", "뤼이드", "에어스메디컬"]

test_state = {
    "current_index": 0,
    "investment_summary_retry_count": 0
}

In [151]:

final_state = graph.invoke(test_state, config={"recursion_limit": 50})

0
1
2
3
4
✅ Markdown 파일 저장 완료: 투자_최종_보고서.md


In [153]:
print(final_state["final_report"])  # 전체 요약 보고서

[최종 종합 보고서]

**투자 우선순위:**

1. **트웰브랩스**
   - 최종 점수: 8.3
   - **투자 추천 여부 및 이유:**
     - 시장성: 의료 분야의 음성 기록 자동화에 대한 수요와 기술력을 바탕으로 높은 시장성을 보이고 있음.
     - 기술력: 딥러닝 기반 음성 인식 엔진의 우수한 성능과 의료 분야에 특화된 알고리즘으로 기술력이 뛰어남.
     - 경쟁력: 모바일 환경에 적합한 모델 경량화로 차별성을 가지고 있지만, 경쟁사들의 기술력과 시장 점유율을 고려하면 경쟁 우위가 부족함.

2. **업스테이지**
   - 최종 점수: 8.3
   - **투자 추천 여부 및 이유:**
     - 시장성: 의료 분야의 음성 기록 자동화에 대한 수요와 독창적인 알고리즘을 통해 높은 시장성을 보임.
     - 기술력: 낮은 지연시간과 높은 인식률을 보이는 딥러닝 기반 음성 인식 엔진으로 기술력이 우수함.
     - 경쟁력: 모델 경량화로 모바일 환경에 적합하나, 경쟁사들의 기술력과 시장 점유율을 고려하면 경쟁 우위가 부족함.

3. **뤼이드**
   - 최종 점수: 8.1
   - **투자 추천 여부 및 이유:**
     - 시장성: 의료 분야의 음성 기록 자동화에 대한 수요와 독창적인 알고리즘을 통해 높은 시장성을 보임.
     - 기술력: 낮은 지연시간과 높은 인식률을 보이는 딥러닝 기반 음성 인식 엔진으로 기술력이 우수함.
     - 경쟁력: 모델 경량화로 모바일 환경에 적합하나, 경쟁사들의 기술력과 시장 점유율을 고려하면 경쟁 우위가 부족함.

**산업/기술 트렌드 분석:**
- 의료 분야의 음성 기록 자동화 기술이 주목받고 있으며, 딥러닝 기술을 활용한 음성 인식 엔진의 발전이 빠르게 이루어지고 있음.
- 모바일 환경에 적합한 경량화 기술이 중요시되며, 의료 분야에서의 활용 가능성이 높아지고 있음.

**공통 리스크 요인:**
- 경쟁사들의 기술력과 시장 점유율이 높아 경쟁이 치열할 수 있음.
- 의료 분야의 규제와 보안

In [154]:
final_state["reports"]

['\n    [업스테이지 보고서]\n\n    A. 기술 분석\n    \n업스테이지는 자체 개발한 딥러닝 기반 음성 인식 엔진을 보유하고 있으며, 경쟁사 대비 낮은 지연시간과 높은 인식률을 보인다는 점에서 기술력이 뛰어나다. 특히 의료 분야의 음성 기록 자동화에 특화된 알고리즘이 독창적이며, 이를 통해 다양한 의료 분야로 확장할 수 있는 가능성이 있다.\n    \n\n    B. 경쟁사 비교\n    \n업스테이지는 주요 경쟁사는 클라우드 인프라와 병원 시스템과의 연동성이 뛰어나지만, 이 스타트업은 모델 경량화로 인해 모바일 환경에 더 적합하다는 점에서 차별성을 갖고 있다. 그러나 경쟁사들의 기술력과 시장 점유율을 고려하면 지속 가능성에 대한 확신은 아직 부족하다.\n    \n\n    C. 시장 분석\n    \n업스테이지는 자체 개발한 딥러닝 기반 음성 인식 엔진을 보유하고 있으며, 경쟁사 대비 낮은 지연시간과 높은 인식률을 보인다는 점에서 기술력이 뛰어나다. 특히 의료 분야의 음성 기록 자동화에 특화된 알고리즘이 독창적이며, 이를 통해 다양한 의료 분야로 확장할 수 있는 가능성이 있다.\n    \n\n    D. 투자 평가\n    1. **시장성 평가**\n   - 점수: 8\n   - 설명: 의료 분야의 음성 기록 자동화에 대한 수요가 높아지고 있으며, 업스테이지의 기술력과 독창적인 알고리즘을 통해 시장 성장 가능성이 크다. 또한, 음성 기록 자동화의 진입장벽이 높아 다른 기업들이 쉽게 진입하기 어렵다.\n\n2. **제품 기술력 평가**\n   - 점수: 9\n   - 설명: 업스테이지의 딥러닝 기반 음성 인식 엔진은 경쟁사 대비 낮은 지연시간과 높은 인식률을 보여주며, 특히 의료 분야에 특화된 알고리즘은 독창적이고 확장성이 높다. 이러한 기술력은 기업의 성장을 뒷받침해줄 것으로 기대된다.\n\n3. **경쟁 우위 평가**\n   - 점수: 7\n   - 설명: 업스테이지는 경쟁사와 비교했을 때 모델 경량화로 모바일 환경에 더 적합하다는 차